In [ ]:
# 这是用RNN模型将sin的输入转换为cos的输出；
import torch
import torch.nn as nn
from torch.autograd import Variable
import numpy as np
import matplotlib.pyplot as plt


In [ ]:
TIME_STEP = 9                               # rnn 定义的时间步数
INPUT_SIZE = 1                              # 因为输入的是1个数字，输入也就是1个feature
LR = 0.01                                   # learning rate
HIDDEN_SIZE = 64                            # 隐藏层的特征数量
torch.manual_seed(1)                        # 随机数种子

In [ ]:

class RNN(nn.Module):                                   # hidden = np.tanh(np.dot(self.W_hh, hidden) + np.dot(self.W_xh, x))
    def __init__(self):
        super().__init__()
        self.rnn = nn.RNN(                              # 自己定义一个RNN神经网络
            input_size = INPUT_SIZE,                    # 输入 x 的feature 维度
            hidden_size = HIDDEN_SIZE,                  # 隐状态 hidden_state 中的feature维度
            num_layers = 5,                             # RNN 的层数
            nonlinearity='relu',                        # 指定激活函数 [‘tanh’ | ’relu’]. 默认: ‘tanh’
            bias = True,                                # 如果是 False , 那么 RNN 层就不会使用偏置权重 b_ih 和 b_hh, 默认: True
            batch_first = True,                         # 如果 True, 输入Tensor的shape应该是(batch, seq, features),并且输出也是一样.
            dropout = 0,                                # 如果值非零, 那么除了最后一层外, 其它层的输出都会套上一个 dropout 层
            bidirectional = False                       # 如果 True , 将会变成一个双向 RNN, 默认为 False
        )
        self.out = nn.Linear(HIDDEN_SIZE, 1)            # 定义一个输出层，这是RNN的最后输出，只用输出output_vector

    def forward(self, x, h_state):                      # 这就是RNN每次输入的参数x和h。RNN输入必需是3维tensor
        # x (batch, time_step, input_size)              # 这是RNN的ｘ的维度            (批量, 序列长度, 输入的特征维度）
        # h_state (n_layers, batch, hidden_size)        # 这是hidden_state的维度       (层数×方向, 批量, 输出的特征维度）/*方向：单向是１；双向是２*/
        # r_out (batch, time_step, hidden_size)         # 这是网络实际输出的r_out的维度 (批量，序列长度，输出的特征维度X方向）
        r_out, h_state = self.rnn(x, h_state)           # RNN每次输入x, hidden_state; 输出r_out, hidden_state; 给定一个序列x,每个x.size=[BATCH_SIZE, feature].
        outs = []
        for time_step in range(r_out.size(1)):
            outs.append(self.out(r_out[:, time_step, :]))

        return torch.stack(outs, dim=1), h_state        # RNN的forward输出了output_vector, hidden_state

In [ ]:
rnn = RNN()
optimizer = torch.optim.Adam(rnn.parameters(), lr=LR, weight_decay=1e-6)    # 定义了优化函数ADAM对RNN的参数反向优化，定义了 learning rate 和 权重衰减率
loss_func = nn.MSELoss()                                                    # 定义了模型的损失函数用 mean-square-error


h_state = None                                                          # 起始时输入给RNN的hidden_state就是None

for step in range(200):                                                # 计算２000次。相当于２000个顺序的时间片数据丢进去计算
    start, end = step*np.pi, (step+1)*np.pi                             # 每次计算的区间就是一个PI里的数据
    steps = np.linspace(start, end, TIME_STEP, dtype=np.float32)        # 在[star，star+PI]之间均匀地取TIME_STEP个点
    x_np = np.sin(steps)                                                # 这就是用来输入的数据。就是每个点的sin值
    y_np = np.cos(steps+np.pi/2)                                          # 这就是需要被预测出的数据。就是每个点的cos值
    
    x = Variable(torch.from_numpy(x_np[np.newaxis, :, np.newaxis]))     # 将x_np从1维(n)转化成3维(1,n,1);forward()里定义了x格式(batch, seq_len, input_size)
    y = Variable(torch.from_numpy(y_np[np.newaxis, :, np.newaxis]))     # 将y_np从1维(n)转化成3维(1,n,1);forward()里定义了out格式(batch, seq_len, input_size)
    # print(x.size())
    # print(y.size())
    
    prediction, h_state = rnn(x, h_state)                               # 这就是一次RNN模型forward()得出来的结果
    # print(h_state.size())                                             # h_state是tensor(data, grad_fn=xxxx)它的data格式为tensor(n_layers, batch, hidden_size)
    h_state = Variable(h_state.data)                                    # 把 h_state(tensor)中的数据取出来,因为h_state包含.data和.grad 舍弃了梯度

    loss = loss_func(prediction, y)                                     # 用损失函数计算误差
    optimizer.zero_grad()                                               # 清空优化器梯度
    loss.backward()                                                     # 反向传播求解梯度
    optimizer.step()                                                    # 更新权重的参数。优化网络参数具体应指 W_xh, W_hh, b_h 以及 W_hq, b_q
    
    if (step+1)%20==0:
        print("step:{}  loss:{:.9f}".format(step, loss.data.float()))

        plt.ion()                                                           # 把实际图和预测图动态打印出来
        plt.plot(steps, y_np, color='b')
        plt.plot(steps, np.squeeze(prediction.data.numpy()), color='r')
        plt.show()
        plt.pause(0.30)

In [ ]:
# 以下为新的 RNN 例子

In [ ]:
# 试一下 RNN 的变长seq的处理情况
import torch
import torch.nn as nn
from torch.nn.utils.rnn import pad_sequence
from torch.nn.utils.rnn import pack_sequence
from torch.nn.utils.rnn import pad_packed_sequence
from torch.nn.utils.rnn import pack_padded_sequence

In [ ]:
# 生成 3 句话
sa = [[1],[2],[3]]
sb = [[4], [5]]
sc = [[6]]

In [ ]:
# 再来定义一个RNN
rnn = nn.RNN(1, 3, batch_first=True)

In [ ]:
# 用rnn处理一下第一句话
input = torch.Tensor([sa])
rnn(input)

In [ ]:
# 把 sa,sb,sc 做成一个batch来处理
# 下面这样处理是错误的
# input = torch.Tensor([sa, sb, sc])
# 因为sa的dim是3，sb的dim是2，sc的dim是1，这不匹配
# [
#  [[1],[2],[3]],
#  [[4],[5]],
#  [[6]]
# ]
# 要把矩阵调整到一样维度的话，做补齐
# [
#  [[1],[2],[3]]
#  [[4],[5],[0]]
#  [[6],[0],[0]]
# ]

# 下面用 0 来补齐，并调整成 batch_first
padded_seq = pad_sequence([torch.Tensor(sa), torch.Tensor(sb), torch.Tensor(sc)], batch_first=True)
print(padded_seq)

# 如果不是 batch_first 的模式
# padded_seq = pad_sequence([torch.Tensor(sa), torch.Tensor(sb), torch.Tensor(sc)], batch_first=False)
# print(padded_seq)

rnn(padded_seq)

In [ ]:
# 或者不再补 0， 直接每次抽取一个元素后生成seq 后再发给RNN
packed_seq = pack_sequence([torch.Tensor(i) for i in [sa, sb, sc]])
print(packed_seq)
# 但是packed_seq默认是seq_first的，所以前面的rnn要重新来
rnn = nn.RNN(1, 3)
rnn(packed_seq)
                            

In [ ]:
################# 下面试试股票数据处理 #################
################ CPU CPU CPU CPU CPU #################

In [ ]:
import pandas as pd

In [ ]:
# 加载数据
dataset = pd.read_csv("000600.csv", index_col=0)
dataset = dataset.drop(['date','prediction'], axis=1)
# print(dataset.columns)
# print(dataset.tail())
dataset['updown'] = dataset['updown']
print(dataset.shape)
# print(dataset.tail())

In [ ]:
# 方法1，直接将数据等分成batch
# 把数据分成训练数据和测试数据
# print(dataset)
# n_train = 120
# train = dataset.iloc[:n_train, :].reset_index(drop=True)
# test  = dataset.iloc[n_train:, :].reset_index(drop=True)

# train_x, train_y = train.iloc[:, 1:], train.iloc[:, 0]
# test_x, test_y = test.iloc[:, 1:], test.iloc[:, 0]

# train_y = np.array(train_y).reshape(-1, 1)
# test_y = np.array(test_y).reshape(-1, 1)

# print(train_x.shape)
# print(train_y.shape)
# print(test_x.shape)
# print(test_y.shape)


In [ ]:
# 数据转换成 tesnsor
# train_x = torch.tensor(np.array(train_x))
# train_y = torch.tensor(np.array(train_y))
# test_x = torch.tensor(np.array(test_x))
# test_y = torch.tensor(np.array(test_y))
# print(train_x.shape)
# print(train_y.shape)
# print(test_x.shape)
# print(test_y.shape)

# # 数据转成 batch 的格式
# batch_count = 8               # 把训练数据切割成 batch_count 个batch
# BATCH_SIZE = 15               # 每个batch里面有 BATCH_SIZE 个数据

# # 把训练数据分成 8 个batch，每个 15 个数据
# train_x = train_x.reshape(batch_count, BATCH_SIZE, 117)
# train_y = train_y.reshape(batch_count, BATCH_SIZE, 1)
# # 把测试数据分成 2 个batch，每个 batch 有 15 个数据
# test_x = test_x.reshape(2, 15, 117)
# test_y = test_y.reshape(2, 15, 1)

In [ ]:
# 方法2，将数据按照BATCH_SIZE的窗口进行滑动，每个窗口数据做一组

# # 数据转成sequence的格式，这里定义每个seq的长度
seq_len = 15
BATCH_SIZE = 2                                                    # 注意：BATCH_SIZE是要能够整除seq_count的

# 把数据切换成 BATCH_SIZE 的一个个batch
rolling_data = pd.DataFrame()
for i in dataset.rolling(seq_len):
    if i.shape[0] == seq_len:
        rolling_data = rolling_data.append(i)

rolling_data = rolling_data.values.reshape(-1, seq_len, 118)                 # 数据一共是 seq_count x seq_len x in_dim

print("rolling_data: {}".format(rolling_data.shape))
print("seq count: {}".format(rolling_data.shape[0]))                 # 所以一共有 seq_count 列数据，每一行的数据是118维 （包括y）
print("seq length: {}".format(seq_len))


total_batch_count = int(rolling_data.shape[0]/BATCH_SIZE)                          # 把数据规划成 batch_count 个 batch

print("total batch count: {}".format(total_batch_count))
print("batch size: {}".format(BATCH_SIZE))

rolling_data = rolling_data.reshape(total_batch_count, BATCH_SIZE, seq_len, 118)  # 把数据转成 total_batch_count x BATCH_SIZE x seq_len x in_dim 格式
rolling_data = torch.tensor(rolling_data)
print("rolling_data: {}".format(rolling_data.shape))


train_batch_count = total_batch_count - 1
test_batch_count = total_batch_count - train_batch_count

train = rolling_data[:train_batch_count, :, :, :]
test  = rolling_data[train_batch_count:, :, :, :]


# train = rolling_data.iloc[:train_batch_count*BATCH_SIZE, :].reset_index(drop=True)
# test  = rolling_data.iloc[train_batch_count*BATCH_SIZE:, :].reset_index(drop=True)

# train_x, train_y = train.iloc[:, 1:], train.iloc[:, 0]
# test_x, test_y = test.iloc[:, 1:], test.iloc[:, 0]

train_x, train_y = train[:,:,:,1:], train[:,:,:,0:1]
test_x,  test_y  = test[:,:,:, 1:],  test[:,:,:,0:1]

# train_y = torch.tensor(np.array(train_y).reshape(-1, 1))
# test_y = torch.tensor(np.array(test_y).reshape(-1, 1))

print("train_x: {}".format(train_x.shape))
print("train_y: {}".format(train_y.shape))
print("test_x:  {}".format(test_x.shape))
print("test_y:  {}".format(test_y.shape))
print("train_batch_count: {}".format(train_batch_count))
print("test_batch_count:  {}".format(test_batch_count))

# # 数据转换成 tesnsor
# train_x = torch.tensor(np.array(train_x))
# train_y = torch.tensor(np.array(train_y))
# test_x = torch.tensor(np.array(test_x))
# test_y = torch.tensor(np.array(test_y))
# print(train_x.shape)
# print(train_y.shape)
# print(test_x.shape)
# print(test_y.shape)

# # batch first
# # 把训练数据分成 ... 个batch，每 batch 有 15 个sequence
# train_x = train_x.reshape(train_batch_count, BATCH_SIZE, seq_len, 117)
# train_y = train_y.reshape(train_batch_count, BATCH_SIZE, seq_len, 1)
# # 把测试数据分成 1 个batch，每个 batch 有 15 个sequence
# test_x = test_x.reshape(test_batch_count, BATCH_SIZE, seq_len, 117)
# test_y = test_y.reshape(test_batch_count, BATCH_SIZE, seq_len, 1)

# seq first
# # 把训练数据分成 ... 个batch，每 batch 有 15 个sequence
# train_x = train_x.reshape(BATCH_SIZE, train_batch_count, 117)
# train_y = train_y.reshape(BATCH_SIZE, train_batch_count, 1)
# # 把测试数据分成 1 个batch，每个 batch 有 15 个sequence
# test_x = test_x.reshape(BATCH_SIZE, test_batch_count, 117)
# test_y = test_y.reshape(BATCH_SIZE, test_batch_count, 1)
# print("train_x: {}".format(train_x.size))
# print("train_y: {}".format(train_y.size))
# print("test_x:  {}".format(test_x.size))
# print("test_y:  {}".format(test_y.size))



In [ ]:
# 开始定义 RNN 模型
TIME_STEP = seq_len                                        # 一般这个单独设定，这里为了简单，还是直接就等于seq_len的方便。其实也就是等于最长的那个sequence length
INPUT_SIZE = 117
HIDDEN_SIZE = 768

class RNN(nn.Module):
    def __init__(self):
        super(RNN, self).__init__()
        
        self.rnn = nn.RNN(
                input_size = INPUT_SIZE,                    # 输入 x 的feature 维度
                hidden_size = HIDDEN_SIZE,                  # 隐状态 hidden_state 中的feature维度
                num_layers = 5,                             # RNN 的层数
                nonlinearity='relu',                        # 指定激活函数 [‘tanh’ | ’relu’]. 默认: ‘tanh’
                bias = True,                                # 如果是 False , 那么 RNN 层就不会使用偏置权重 b_ih 和 b_hh, 默认: True
                batch_first = True,                         # 如果 True, 输入Tensor的shape应该是(BATCH_SIZE, seq_len, features),并且输出也是一样.
                dropout = 0.1,                                # 如果值非零, 那么除了最后一层外, 其它层的输出都会套上一个 dropout 层
                bidirectional = False                       # 如果 True , 将会变成一个双向 RNN, 默认为 False
        )
        
        self.out = nn.Linear(HIDDEN_SIZE, 1)
        
    def forward(self, x, h_state):
        r_out, h_state = self.rnn(x, h_state)
        outs = []
        for time_step in range(TIME_STEP):
            outs.append(self.out(r_out[:, time_step, :]))

        return torch.stack(outs, dim=1), h_state

In [ ]:
# 初始化 RNN 模型
rnn = RNN()
LR = 5e-5
loss_func = nn.MSELoss()
optimizer = torch.optim.Adam(rnn.parameters(), lr=LR, weight_decay=1e-8)

i = 0
h_state = None
# 训练 RNN 模型; 计划训练 20 轮
for step in range(train_batch_count*20):
    rnn = rnn.double()
    pred, h_state = rnn(train_x[i], h_state)
    h_state = h_state.data
    loss = loss_func(pred, train_y[i])
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    
    if (i+1)%train_batch_count == 0:
        print(loss)
        i=0
    else:
        i=i+1

In [ ]:
# 用模型来做测试数据的误差
h_state = None
pred, h_state = rnn(test_x[0], h_state)
loss = loss_func(pred, test_y[0])
print(loss)

actual_line = np.array(test_y[0]).flatten().tolist()
pred_line = pred.data.numpy().flatten().tolist()

print(len(actual_line) == len(pred_line))
plt.plot(actual_line, 'r--')
plt.plot(pred_line, 'b-')
plt.show()

In [ ]:
################ GPU GPU GPU GPU GPU #################
################ GPU GPU GPU GPU GPU #################
################ GPU GPU GPU GPU GPU #################

In [ ]:
# GPU -- 准备来做一个GPU版本的RNN
import torch
import torch.nn as nn
from torch.autograd import Variable
import numpy as np
import pandas as pd
import time
import matplotlib.pyplot as plt

# 设置 GPU 优先
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

# 加载数据
dataset = pd.read_csv("601229.csv", index_col=0)
dataset = dataset.drop(['date','prediction'], axis=1)
# print(dataset.columns)
# print(dataset.tail())
dataset['future'] = dataset['future']
print(dataset.shape)
# print(dataset.tail())

# 方法2，将数据按照BATCH_SIZE的窗口进行滑动，每个窗口数据做一组

# # 数据转成sequence的格式，这里定义每个seq的长度
seq_len = 45
BATCH_SIZE = 1                                                    # 注意：BATCH_SIZE是要能够整除seq_count的

# 把数据切换成 BATCH_SIZE 的一个个batch
rolling_data = pd.DataFrame()
for i in dataset.rolling(seq_len):
    if i.shape[0] == seq_len:
        rolling_data = rolling_data.append(i)

rolling_data = rolling_data.values.reshape(-1, seq_len, dataset.shape[1])                 # 数据一共是 seq_count x seq_len x (in_dim+1)

print("rolling_data shape: {}".format(rolling_data.shape))
print("seq count: {}".format(rolling_data.shape[0]))                                       # 所以一共有 seq_count 列数据，每一行的数据是123维 （包括y）
print("seq length: {}".format(seq_len))


total_batch_count = int(rolling_data.shape[0]/BATCH_SIZE)                                   # 把数据规划成 batch_count 个 batch

print("total batch count: {}".format(total_batch_count))
print("batch size: {}".format(BATCH_SIZE))

rolling_data = rolling_data.reshape(total_batch_count, BATCH_SIZE, seq_len, dataset.shape[1])  # 把数据转成 total_batch_count x BATCH_SIZE x seq_len x in_dim 格式
rolling_data = torch.tensor(rolling_data)
print("rolling_data: {}".format(rolling_data.shape))


train_batch_count = total_batch_count - 1
test_batch_count = total_batch_count - train_batch_count

train = rolling_data[:train_batch_count, :, :, :]
test  = rolling_data[train_batch_count:, :, :, :]

train_x, train_y = train[:,:,:,1:], train[:,:,:,0:1]
test_x,  test_y  = test[:,:,:, 1:],  test[:,:,:,0:1]

train_x = train_x.to(device)
train_y = train_y.to(device)
test_x = test_x.to(device)
test_y = test_y.to(device)

print("train_x: {}".format(train_x.shape))
print("train_y: {}".format(train_y.shape))
print("test_x:  {}".format(test_x.shape))
print("test_y:  {}".format(test_y.shape))
print("train_batch_count: {}".format(train_batch_count))
print("test_batch_count:  {}".format(test_batch_count))

In [ ]:
# GPU -- 开始定义 RNN 模型
TIME_STEP = seq_len                                        # 一般这个单独设定，这里为了简单，还是直接就等于seq_len的方便。其实也就是等于最长的那个sequence length
INPUT_SIZE = 122
HIDDEN_SIZE = 768
NUM_LAYERS = 8

np.random.seed(3)
torch.manual_seed(3)
torch.cuda.manual_seed(3)
torch.backends.cudnn.deterministic = True

class RNN(nn.Module):
    def __init__(self):
        super(RNN, self).__init__()
        
        self.dropout = nn.Dropout(p=0.5)
        
        self.rnn = nn.RNN(
                input_size = INPUT_SIZE,                    # 输入 x 的feature 维度
                hidden_size = HIDDEN_SIZE,                  # 隐状态 hidden_state 中的feature维度
                num_layers = NUM_LAYERS,                    # RNN 的层数
                nonlinearity='relu',                        # 指定激活函数 [‘tanh’ | ’relu’]. 默认: ‘tanh’
                bias = True,                                # 如果是 False , 那么 RNN 层就不会使用偏置权重 b_ih 和 b_hh, 默认: True
                batch_first = True,                         # 如果 True, 输入Tensor的shape应该是(BATCH_SIZE, seq_len, features),并且输出也是一样.
                dropout = 0.5,                              # 如果值非零, 那么除了最后一层外, 其它层的输出都会套上一个 dropout 层
                bidirectional = False                       # 如果 True , 将会变成一个双向 RNN, 默认为 False
        )
        
        self.fc_out = nn.Linear(HIDDEN_SIZE, 1)
        
        # weight的参数用kaiming初始化；bias用zeros初始化；
        for name, param in self.rnn.named_parameters():
            if name.startswith("weight"):
                nn.init.kaiming_normal_(param, nonlinearity='relu')
            else:
                nn.init.zeros_(param)

        # weight的参数用xavier初始化；bias用zeros初始化；
        # for name, param in self.rnn.named_parameters():
        #     if name.startswith("weight"):
        #         nn.init.xavier_normal_(param)
        #     else:
        #         nn.init.zeros_(param)
                
            
    def forward(self, x):
        x = self.dropout(x)
        
        h_state = torch.zeros(NUM_LAYERS, BATCH_SIZE, HIDDEN_SIZE).double().to(device)
        r_out, h_state = self.rnn(x, h_state)                         # compute from [1, seq_len, input_dim] to [1, seq_len, hidden_dim] 

        outs = []
        
        for time_step in range(TIME_STEP):
            out = self.fc_out(r_out[:, time_step, :])                 # compute from [1, 1, hidden_dim] to [output_dim]
            outs.append(out)                                          # Add all seq_len data to list like : [[pred1, pred2,...,predn]]
        
        return torch.stack(outs, dim=1)                               # conver from [[pred1,...,predn]] to [[[pred1], ... ,[prdn]]]


In [ ]:
%%time
# GPU -- 初始化 RNN 模型
model = RNN()
model.double().to(device)
LR = 1e-5
epoches = 500

epoch_loss_list = []
epoch_loss = 0
lr_list = []



# loss_func = nn.L1Loss()
loss_func = nn.MSELoss(reduction="mean")
optimizer = torch.optim.Adam(model.parameters(), lr=LR, weight_decay=1e-3)
# scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=epoches, eta_min=0, last_epoch=-1)
scheduler = torch.optim.lr_scheduler.ExponentialLR(optimizer, gamma=0.995, last_epoch=-1)


# 训练 RNN 模型; 计划训练 100 轮
for epoch in range(epoches):
    for step in range(train_batch_count):
        pred = model(train_x[step])
        pred.to(device)
        # h_state = h_state.data.to(device)

        loss = loss_func(pred, train_y[step])
        optimizer.zero_grad()
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=20, norm_type=2)
        optimizer.step()
        
        epoch_loss += loss.data
    

    if (epoch+1)%10 == 0:
        print("{} of {} epoches get epoch_loss: {:.6f}".format(epoch, epoches, epoch_loss))
        lr_list.append(optimizer.state_dict()['param_groups'][0]['lr'])
        epoch_loss_list.append(epoch_loss.cpu())
        # print(optimizer.state_dict()['param_groups'][0]['lr'])
        # for p in optimizer.param_groups:
        #     p['lr'] *= 0.9
        print(scheduler.get_last_lr())
    
    scheduler.step()    

    epoch_loss = 0

plt.plot(lr_list)
plt.show()
                   
plt.plot(epoch_loss_list)
plt.show()

In [ ]:
# GPU -- 用模型来做测试数据的误差

for step in range(test_batch_count):
    pred = model(test_x[step])

    loss = loss_func(pred, test_y[step].to(device))

    print("Prediction: {:.2f}".format(float(pred[-1][-1].data)))
    
    print("loss : {:.6f}".format(float(loss.data)))

    # print(test_y[step][-1].flatten().shape)
    # print(pred[-1].flatten().shape)
    
    # actual_line = test_y[step][-1].cpu().detach().flatten().numpy()
    # pred_line   = pred[-1].cpu().detach().flatten().numpy()
    
    actual_line = test_y[step].cpu().detach().flatten().numpy()
    pred_line   = pred.cpu().detach().flatten().numpy()
    
    plt.plot(actual_line, 'r--')
    plt.plot(pred_line, 'b-')
    plt.show()
    
# actual_line = np.array(test_y[-1]).flatten().tolist()
# pred_line = pred.data.numpy().flatten().tolist()

# actual_line = np.array(test_y[0][-1]).flatten().tolist()
# pred_line = pred[-1].data.numpy().flatten().tolist()


